<a id='0-section'></a>
##### I. Разведочный анализ данных:
* [1. Изучение файлов с данными, получение общей информации](#1-section)
* [2. Исследовательский анализ данных](#2-section)

##### II. Подготовка данных к обучению:
* [1. Подготовка признаков](#3-section)
* [2. Разбиение на выборки](#4-section)

##### III. Исследование моделей классификации:
* [1. Константная модель](#5-section)
* [2. Дисбаланс классов](#6-section)
* [3. Баланс классов](#7-section)
* [4. Изменение порога](#8-section)
* [5. Визуализация метрик](#9-section)

##### [IV. Общий вывод](#10-section)

## I. Разведочный анализ данных

<a id='1-section'></a>
### 1. Изучение файлов с данными, получение общей информации

[Вернуться к оглавлению](#0-section)

In [1]:
# загрузим необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    recall_score, 
    precision_score, 
    f1_score, 
    precision_recall_curve, 
    roc_auc_score, 
    roc_curve,
    make_scorer
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [2]:
# создадим функцию для загрузки датафрейма
def get_df(data):
    try:
        df = pd.read_csv(
            "C:/Users/79090/YandexDisk/from asus/MyStudy/Data Science/Яндекс.Практикум/Самостоятельные проекты/Обучение с учителем/customer_churn/{}.csv".format(
                data
            ),
            index_col="CustomerId",
        )
    except:
        print("Ошибка при чтении файла")
    df.columns = df.columns.str.lower()
    print("Несколько строк из датафрейма")
    print()
    display(df.sample(5))
    print()
    print("Общая информация о датафрейме")
    print()
    print(df.info())
    return df

In [3]:
churn = get_df('Churn')

Несколько строк из датафрейма



,rownumber,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
CustomerId,,,,,,,,,,,,,
15800814,3511,Palerma,534,France,Male,35,NaN,81951.74,2,1,0,115668.53,0
15780835,1674,Liang,652,Germany,Female,26,1.0,131908.35,1,1,1,179269.79,0
15747265,2768,Huang,598,Germany,Female,27,10.0,171283.91,1,1,1,84136.12,0
15743149,6801,Findlay,711,France,Female,35,8.0,0.00,1,1,1,67508.01,0
15637414,741,Gell,618,France,Female,24,7.0,128736.39,1,0,1,37147.61,0



Общая информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 15634602 to 15628319
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   surname          10000 non-null  object 
 2   creditscore      10000 non-null  int64  
 3   geography        10000 non-null  object 
 4   gender           10000 non-null  object 
 5   age              10000 non-null  int64  
 6   tenure           9091 non-null   float64
 7   balance          10000 non-null  float64
 8   numofproducts    10000 non-null  int64  
 9   hascrcard        10000 non-null  int64  
 10  isactivemember   10000 non-null  int64  
 11  estimatedsalary  10000 non-null  float64
 12  exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1.1+ MB
None


Tenure - это количество недвижимости у клиента. Значит "tenure" должен иметь тип данных int64. Также в данном столбце присутствуют пропуски.

Пропуски могли возникнуть из-за разных причин. Данные могли затереться при копировании/скачивании/форматировании. А может клиенты специально не указали данные о количестве своей недвижимости.

<a id='2-section'></a>
### 2. Исследовательский анализ данных

In [ ]:
# подробнее изучим данные
# создадим функцию, возвращающую детальную информацию о данных по столбцу
def get_column_info(data, column):
    print('{: ^}'.format("_" * (len("Числовое описание данных столбца") + len(column) + 1)))
    print()
    print('Числовое описание данных столбца "{}"'.format(column))
    print()
    if data[column].isnull().sum() > 0:
        print('Количество пропусков: {}'.format(data[column].isnull().sum()))
    else:
        print('В столбце нет пропусков')
    print()
    try:
        print(
            "Коэффициент корреляции Пирсона с целевым признаком: {:.2f}".format(
                data[column].corr(
                    data["exited"]
                )
            )
        )
    except:
        pass
    print()
    print(data[column].describe())
    print()
    # выведем max, min, наиболее и наименее частотные значения столбца
    # если уникальных значений больше пяти
    if len(data[column].unique()) > 5:
        if data[column].value_counts().min() != data[column].value_counts().max():
            print('Наиболее частотные значения столбца')
            print()
            print(data[column].value_counts().head())
            print()
            print('Наименее частотные значения столбца')
            print()
            print(data[column].value_counts().tail())
            print()
        print()
        if data[column].dtype in ['int64', 'float64']:
            if data.groupby(column)[column].count().max() != data.groupby(column)[column].count().min():
                print('Максимальные значения столбца')
                print()
                print(data.groupby(column)[column].count()[::-1].head())
                print()
                print('Минимальные значения столбца')
                print()
                print(data.groupby(column)[column].count().head()[::-1])
                print()
            print("Диаграмма размаха столбца", column)
            sns.boxplot(x=data[column])
            plt.show()
            print()
            print("Гистограмма для столбца", column)
            ax = sns.distplot(data[column])
            plt.show()
    else:
        print()
        print('Распределение данных столбца "{}"'.format(column))
        print()
        print(data[column].value_counts())
        data[column].value_counts().plot(kind='pie', subplots=True, figsize=(9,6), autopct='%1.1f%%')
        plt.show()

In [ ]:
# создадим функцию для вывода информации о всех столбцах датафрейма
def get_all_columns_info(data):
    for column in data.columns:
        get_column_info(data, column)

In [ ]:
get_all_columns_info(churn)

Кредитный рейтинг имеет нормальное распределение. Заметим всплеск на значении 850. Это максимально возможное значение рейтинга. То есть 233 человека являются максимально добропорядочными с точки зрения кредитного доверия.

Данные по возрасту немного скошены вправо. Самому юному клиенту 18 лет, самому возрастному - 92 года. Возраст большинства клиентов находится в диапазоне 31 - 44 лет.

3617 человек имеют баланс равный 0. В остальных случаях распределение выглядит нормальным.

Посмотрим на тех, кто отказался от услуг компании

In [ ]:
left = churn.query('exited == 1')

In [ ]:
get_all_columns_info(left) 

In [ ]:
# посмотрим на текущих клиентов
current = churn.query('exited == 0')
get_all_columns_info(current) 

Сравнивая полученную информацию об ушедших клиентах, можно сказать об основных изменениях (в скобках представлены значения для текущих клиентов):
- средний возраст ушедших клиентов - `45`лет (`37.4`);
- средний баланс на карте у ушедших клиентов - `91108` (`72745`);

In [ ]:
# построим матрицу корреляции всего df
churn.corr().style.background_gradient(cmap='PuBuGn')

Присутствует слабая прямая корреляция между возрастом и фактом ухода (0.285).

### Вывод

Изучили общую информацию о датафрейме:
 - названия столбцов привели к нижнему регистру;
 - в столбце с количеством недвижимости "tenure" присутствуют пропуски;

Также был проведён исследовательский анализ данных. В результате чего выяснилось, что средний возраст ушедших клиентов `45`лет, а действующих клиентов - `37.4`; средний баланс на карте у ушедших клиентов - `91108`, а у текущих клиентов - `72745`.

## II. Подготовка данных к обучению

<a id='3-section'></a>
### 1. Подготовка признаков

[Вернуться к оглавлению](#0-section)

In [ ]:
# заменим пропуски в столбце tenure на характерные
# для этого разделим значения кредитного рейтинга и возраста клиентов на группы
def get_group_creditscore(row):
    creditscore = row['creditscore']
    return creditscore // 50
def get_group_age(row):
    age = row['age']
    return age // 10

In [ ]:
# создадим копию df
churn_up = churn.copy()

In [ ]:
# заполним значения с помощью функции apply(), get_group_creditscore(), get_group_age()
churn_up['creditscore_level'] = churn_up.apply(get_group_creditscore, axis=1)
churn_up['age_level'] = churn_up.apply(get_group_age, axis=1)

In [ ]:
# создадим функцию, которая по заданному df и столбцу, создавала бы сводную таблицу, в которой для каждой возрастной категории и 
# категории кредитного рейтинга считала бы среднее значение выбранного столбца
# и затем для каждого соответствия категорий возраста и кредитного рейтинга исходного df заполняла бы пропуск в данном 
# столбце из этой сводной таблицы
def fill_with_pivot_table(data, column):
    # создадим сводную таблицу, в которой для каждой марки и модели автомобиля считается среднее значение столбца column
    pivot_table = data.pivot_table(index=['creditscore_level', 'age_level'], values=column).astype(int)
    # создадим df, который является срезом data, состоящим из пропусков в столбце column
    df_nan = data.loc[data[column].isnull()]
    def change_values(row):
        creditscore_level = row['creditscore_level']
        age_level = row['age_level']
        # создадим конструкцию try/except на случай, если в pivot_table не найдется значения по двум категориям
        try:
            value = pivot_table.loc[(creditscore_level, age_level),:]
        except:
            value = np.nan
        return value    
    # заполним пропуски в df_nan с помощью функции change_values() и apply()  
    df_nan = df_nan.copy()
    df_nan[column] = df_nan.apply(change_values, axis=1)
    # теперь надо заменить значения в data из df_nan
    for i in df_nan.index:
        if i in data.index:
            data.loc[i,column] = df_nan.loc[i,column]

In [ ]:
# применим функцию fill_with_pivot_table()
fill_with_pivot_table(churn_up, 'tenure')

In [ ]:
# посмотрим на пропуски
churn_up.loc[churn_up['tenure'].isnull()]

В данных остался только 1 пропуск. Мужчина 92 лет. Получим медианное значение для категории creditscore_level = 15 и возрастом не менее 70 лет.

In [ ]:
round(churn_up.query('creditscore_level == 15 & age_level > 6')['tenure'].median())

In [ ]:
# заменим пропуск на данное значение
churn_up = churn_up.fillna(
    round(churn_up.query("creditscore_level == 15 & age_level > 6")["tenure"].median())
)

In [ ]:
# изменим тип данных в "tenure" на целочисленный
churn_up['tenure'] = churn_up['tenure'].astype('int64')

Но вначале воспользуемся библиотекой LOFO.

In [ ]:
pip install lofo-importance

In [ ]:
from tqdm.autonotebook import tqdm
from lofo import LOFOImportance, Dataset, plot_importance

In [ ]:
target="exited"

In [ ]:
dataset = Dataset(
    df=churn_up,
    target="exited",
    features=[col for col in churn_up.columns if col != target],
)

In [ ]:
# определим схему кроссвалидации и метрику
lofo_imp = LOFOImportance(dataset, cv=5, scoring="f1")

In [ ]:
# получить среднее значение и стандартное отклонение значений
importance_df = lofo_imp.get_importance()

In [ ]:
# построим график важности признаков
plot_importance(importance_df, figsize=(12, 20))

In [ ]:
# оставим значимые признаки
final = churn_up.drop([
    'age_level',
    'gender',
    'creditscore_level',
    'surname',
], axis=1).copy()

In [ ]:
# выделим столбцы с числовыми и категориальными признаками
numeric = [col for col in final.columns if final[col].dtype in ['int64', 'float64'] and col != target]
category = [col for col in final.columns if final[col].dtype in ['object']]

<a id='4-section'></a>
### 2. Разбиение на выборки

[Вернуться к оглавлению](#0-section)

Необходимо преобразовать категориальные признаки в численные. В этом нам поможет One-Hot Encoding (OHE) и OrdinalEncoder. Учтём также, что факт ухода клиента не зависит от его фамилии, поэтому не будем работать с данным категориальным признаком. Также, как и признаки "rownumber".

In [ ]:
def dummies(data):
    data = pd.get_dummies(data, drop_first=True)
    return data

In [ ]:
# воспользуемся функцией get_dummies()
final_ohe = dummies(final)

In [ ]:
# используем OrdinalEncoder() для преобразования категориальных признаков в числовые
final_oe = final.copy()
enc = OrdinalEncoder()
enc.fit(final[category])
final_oe[category] = pd.DataFrame(enc.transform(final[category]), columns=final[category].columns, index=final[category].index)
final_oe.head()

Все оставшиеся признаки значимы. Но в данных есть значения предполагаемой зарплаты - порядка сотен тысяч, а есть кредитный рейтинг - порядка нескольких сотен. Необходимо масштабировать признаки. Воспользуемся стандартизацией данных.

In [ ]:
def get_features_target(data):
    features = data.drop('exited', axis=1)
    target = data['exited']
    features_train, features_test, target_train, target_test = train_test_split(
        features, target, test_size=0.2, random_state=42, stratify=target)
    print(features_train.shape, features_test.shape, target_train.shape, target_test.shape)
    return features_train, features_test, target_train, target_test

In [ ]:
##
fea_train_ohe, fea_test_ohe, tar_train_ohe, tar_test_ohe = get_features_target(final_ohe)

In [ ]:
##
fea_train_oe, fea_test_oe, tar_train_oe, tar_test_oe = get_features_target(final_oe)

In [ ]:
def get_scaler(features_train, features_test):
    # создадим объект структуры данных StandardScaler
    scaler = StandardScaler()
    # сделаем копии выборок
    features_train = features_train.copy()
    features_test = features_test.copy()
    # настроим на обучающей выборке
    scaler.fit(features_train.loc[:, numeric])
    # преобразуем обучающую и валидационную выборки
    features_train.loc[:, numeric] = scaler.transform(features_train.loc[:, numeric])
    features_test.loc[:, numeric] = scaler.transform(features_test.loc[:, numeric])
    display(features_train.head(3))
    return features_train, features_test

In [ ]:
fea_train_ohe, fea_test_ohe = get_scaler(fea_train_ohe, fea_test_ohe)

In [ ]:
fea_train_oe, fea_test_oe = get_scaler(fea_train_oe, fea_test_oe)

### Вывод

На данном этапе работы мы занимались обработкой и подготовкой данных.

В результате чего
- добавили два столца - категории кредитного рейтинга и возраста. На их основании заменили пропуски в столбце "tenure";
- построили график важности признаков и оставили наиболее значимые признаки;
- выделили столбцы с числовыми и категориальными признаками;
- преобразовали категориальные признаки в числовые с помощью OHE и OrdinalEncoder;
- разбили данные на обучающую и тестовую выборки;
- масштабировали числовые признаки.

Данные подготовлены к дальнейшему обучению.

## III. Исследование моделей классификации

В данной работе будут исследованы три классические модели задачи классификации и три модели градентного бустинга: 
- логистическая регрессия;
- решающее дерево;
- случайный лес;
- LGBMClassifier;
- XGBClassifier;
- CatBoostClassifier.

In [ ]:
#посмотрим, насколько часто клиенты уходят из банка
final['exited'].value_counts(normalize=True).plot(kind='bar', title='Доля клиентов, ушедших из банка')
plt.show()

Заметим, что клиентов, оставшихся в банке, примерно в 4 раза больше ушедших клиентов. Достаточно большая разница.

In [ ]:
f1_scorer = make_scorer(f1_score)

In [ ]:
# напишем функцию для моделей с использованием GridSearchCV()
def get_best_model(model, parameters, x_train, x_test, y_train, y_test):
    clf = GridSearchCV(estimator=model, param_grid=parameters, cv=5, verbose=1, n_jobs=-1, scoring=f1_scorer)
    clf.fit(x_train, y_train)
    best_model = clf.best_estimator_
    best_model.fit(x_train, y_train)
    pred_train = best_model.predict(x_train)
    pred_test = best_model.predict(x_test)
    print('Лучшие параметры модели:', clf.best_params_)
    print()
    print('Матрица ошибок:')
    sns.heatmap(confusion_matrix(y_train, pred_train), annot=True, fmt="d")
    plt.show()
    print()
    # вычислим значение полноты, точности, F1-меры модели
    accuracy_train = accuracy_score(y_train, pred_train)
    recall_train = recall_score(y_train, pred_train)
    precision_train = precision_score(y_train, pred_train)
    f1_train = f1_score(y_train, pred_train)
    accuracy_test = accuracy_score(y_test, pred_test)
    recall_test = recall_score(y_test, pred_test)
    precision_test = precision_score(y_test, pred_test)
    f1_test = f1_score(y_test, pred_test)
    # вычислим значение AUC-ROC
    probabilities_train = best_model.predict_proba(x_train)
    probabilities_one_train = probabilities_train[:, 1]
    roc_auc_train = roc_auc_score(y_train, probabilities_one_train)
    probabilities_test = best_model.predict_proba(x_test)
    probabilities_one_test = probabilities_test[:, 1]
    roc_auc_test = roc_auc_score(y_test, probabilities_one_test)
    print('"ROC-AUC" на обучающей выборке: {}'.format(roc_auc_train))
    print('"ROC-AUC" на тестовой выборке: {}'.format(roc_auc_test))
    df = pd.DataFrame(data=[[accuracy_train,accuracy_test], 
                            [recall_train,recall_test], 
                            [precision_train,precision_test], 
                            [f1_train,f1_test], 
                            [roc_auc_train,roc_auc_test]], 
                      index=['accuracy', 'recall', 'precision', 'f1', 'roc_auc'], 
                      columns=['Метрики на обучающей выборке','Метрики на тестовой выборке'])
    display(df)
    return best_model, df, probabilities_one_test

<a id='5-section'></a>
### 1. Константная модель

[Вернуться к оглавлению](#0-section)

In [ ]:
# определим точность предсказания константной модели
def dummy_model(x_train, x_test, y_train, y_test, constant):
    # константная модель, всегда предсказывающая значение по выбранной стратегии
    dummy = DummyClassifier(strategy='constant', constant=constant, random_state=42)
    dummy.fit(x_train, y_train)
    pred_test = dummy.predict(x_test)
    # вычислим значение точности, полноты, F1-меры и AUC-ROC константной модели
    accuracy = accuracy_score(y_test, pred_test)
    recall = recall_score(y_test, pred_test)
    precision = precision_score(y_test, pred_test)
    f1 = f1_score(y_test, pred_test)
    dummy_probabilities_test = dummy.predict_proba(x_test)
    dummy_probabilities_one_test = dummy_probabilities_test[:, 1]
    auc_roc = roc_auc_score(y_test, dummy_probabilities_one_test)
    df = pd.DataFrame(data=[[accuracy], 
                            [recall], 
                            [precision], 
                            [f1], 
                            [auc_roc]], 
                      index=['accuracy', 'recall', 'precision', 'f1', 'roc_auc'], 
                      columns=['Значения метрик'])
    return df

In [ ]:
dummy_0 = dummy_model(fea_train_ohe, fea_test_ohe, tar_train_ohe, tar_test_ohe, 0)
dummy_0

In [ ]:
dummy_1 = dummy_model(fea_train_ohe, fea_test_ohe, tar_train_ohe, tar_test_ohe, 1)
dummy_1

Лучший результат метрики "ROC-AUC" константной модели - `0.5`.

<a id='6-section'></a>
### 2. Дисбаланс классов

[Вернуться к оглавлению](#0-section)

In [ ]:
# создадим словарь для логистической регрессии
param_log_reg = {
    "C": list(range(1, 100, 1)),
    "random_state": [42],
}

def log_reg(features_train, features_test, target_train, target_test, class_weight):
    # используем функцию get_best_model()
    lr_model, lr_df, lr_proba = get_best_model(
        LogisticRegression(solver="liblinear", class_weight=class_weight),
        param_log_reg,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return lr_model, lr_df, lr_proba

In [ ]:
%%time
lr_model_disb, lr_df_disb, lr_proba_disb = log_reg(
    fea_train_ohe,
    fea_test_ohe, 
    tar_train_ohe,
    tar_test_ohe,
    None
)

In [ ]:
# создадим словарь для решающего дерева
param_dec_tree = {
    'max_depth'         : list(range(1, 20, 1)),
    'min_samples_split' : list(range(2, 5, 1)),
    'min_samples_leaf'  : list(range(1, 5, 1)), 
    'random_state'      : [42],
}
# используем функцию get_best_model()
def dec_tree(features_train, features_test, target_train, target_test, class_weight):
    # используем функцию get_best_model()
    dt_model, dt_df, dt_proba = get_best_model(
        DecisionTreeClassifier(class_weight=class_weight),
        param_dec_tree,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return dt_model, dt_df, dt_proba

In [ ]:
%%time
dt_model_disb, dt_df_disb, dt_proba_disb = dec_tree(
    fea_train_oe,
    fea_test_oe, 
    tar_train_oe,
    tar_test_oe,
    None
)

In [ ]:
# создадим словарь для случайного леса
param_rand_for = {
    'n_estimators'      : list(range(1, 120, 5)),
    'max_depth'         : list(range(1, 15, 1)), 
    'random_state'      : [42],
}
# используем функцию get_best_model()
def rand_for(features_train, features_test, target_train, target_test, class_weight):
    # используем функцию get_best_model()
    rand_for_model, rf_df, rf_proba = get_best_model(
        RandomForestClassifier(class_weight=class_weight),
        param_rand_for,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return rand_for_model, rf_df, rf_proba

In [ ]:
%%time
rf_model_disb, rf_df_disb, rf_proba_disb = rand_for(
    fea_train_oe,
    fea_test_oe, 
    tar_train_oe,
    tar_test_oe,
    None
)

In [ ]:
# создадим словарь для xgboost
param_xgb = {
    "n_estimators": list(range(40, 120, 5)),
    "max_depth": list(range(1, 5, 1)),
    'objective':['binary:logistic'],
    'gamma':[0,0.2],
    'eta':[0.1,0.07,0.06],
}
# используем функцию get_best_model()
def xgb(features_train, features_test, target_train, target_test, class_weight):
    xgb_model, xgb_df, xgb_proba = get_best_model(
        XGBClassifier(use_label_encoder=False, verbosity = 0, random_state=42, silent=True, class_weight=class_weight),
        param_xgb,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return xgb_model, xgb_df, xgb_proba

In [ ]:
%%time
xgb_model_disb, xgb_df_disb, xgb_proba_disb = xgb(
    fea_train_oe,
    fea_test_oe, 
    tar_train_oe,
    tar_test_oe,
    None
)

In [ ]:
# создадим словарь для lightgbm
param_lgb = {
    "n_estimators": list(range(72, 150, 3)),
    "max_depth": list(range(1, 20, 1)),
}
# используем функцию get_best_model()
def lgb(features_train, features_test, target_train, target_test, class_weight):
    lgb_model, lgb_df, lgb_proba = get_best_model(
        LGBMClassifier(
            boosting_type="gbdt",
            verbosity=0,
            objective="binary",
            feature_fraction=0.9,
            random_state=42,
            silent=True,
            class_weight=class_weight,
        ),
        param_lgb,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return lgb_model, lgb_df, lgb_proba

In [ ]:
%%time
lgb_model_disb, lgb_df_disb, lgb_proba_disb = lgb(
    fea_train_oe,
    fea_test_oe, 
    tar_train_oe,
    tar_test_oe,
    None
)

In [ ]:
# создадим словарь для catboost
param_ctb = {
    "depth": list(range(1, 16, 1)),
}
# используем функцию get_best_model()
def ctb(features_train, features_test, target_train, target_test):
    ctb_model, ctb_df, ctb_proba = get_best_model(
        CatBoostClassifier(learning_rate=0.05, 
                           iterations=40, 
                           verbose=False),
        param_ctb,
        features_train,
        features_test,
        target_train,
        target_test,
    )
    return ctb_model, ctb_df, ctb_proba

In [ ]:
%%time
ctb_model_disb, ctb_df_disb, ctb_proba_disb = ctb(
    fea_train_oe,
    fea_test_oe, 
    tar_train_oe,
    tar_test_oe
)

<a id='7-section'></a>
### 3. Баланс классов

[Вернуться к оглавлению](#0-section)

Подготовим данные, чтобы они стали более сбалансированными.

In [ ]:
## используем технику upsampling увеличения выборки
def upsample(features, target):
    # разделим обучающую выборку на отрицательные и положительные объекты
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    # скопируем несколько раз положительные объекты, создаем новую обучающую выборку
    features_upsampled = pd.concat([features_zeros] + [features_ones] * round(len(features_zeros) / len(features_ones)))
    target_upsampled = pd.concat([target_zeros] + [target_ones] * round(len(features_zeros) / len(features_ones)))
    # перемешиваем данные
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled

In [ ]:
## получили новую обучающую выборку
fea_train_up_ohe, tar_train_up_ohe = upsample(fea_train_ohe, tar_train_ohe)
fea_train_up_oe, tar_train_up_oe = upsample(fea_train_oe, tar_train_oe)

In [ ]:
%%time
lr_model_up, lr_df_up, lr_proba_up = log_reg(
    fea_train_up_ohe,
    fea_test_ohe, 
    tar_train_up_ohe,
    tar_test_ohe,
    None
)

In [ ]:
lr_model_bal, lr_df_bal, lr_proba_bal = log_reg(
    fea_train_up_ohe,
    fea_test_ohe, 
    tar_train_up_ohe,
    tar_test_ohe,
    'balanced',
)

In [ ]:
%%time
dt_model_up, dt_df_up, dt_proba_up = dec_tree(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    None
)

In [ ]:
%%time
dt_model_bal, dt_df_bal, dt_proba_bal = dec_tree(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    'balanced'
)

In [ ]:
%%time
rf_model_up, rf_df_up, rf_proba_up = rand_for(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    None
)

In [ ]:
%%time
rf_model_bal, rf_df_bal, rf_proba_bal = rand_for(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    'balanced'
)

In [ ]:
%%time
xgb_model_up, xgb_df_up, xgb_proba_up = xgb(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    None
)

In [ ]:
%%time
xgb_model_bal, xgb_df_bal, xgb_proba_bal = xgb(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    'balanced'
)

In [ ]:
%%time
lgb_model_up, lgb_df_up, lgb_proba_up = lgb(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    None
)

In [ ]:
%%time
lgb_model_bal, lgb_df_bal, lgb_proba_bal = lgb(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe,
    'balanced'
)

In [ ]:
%%time
ctb_model_up, ctb_df_up, ctb_proba_up = ctb(
    fea_train_up_oe,
    fea_test_oe, 
    tar_train_up_oe,
    tar_test_oe
)

Лучшее значение F1-меры оказалось равным `0.6455`. Это значение удалось достичь для модели LightGBM с помощью техники upsampling. Техника downsampling для LightGBM дала результат `0.6204` и для CatBoost - `0.6186`.

Случайный лес показал результат немного хуже - `0.6213`.

Решающее дерево и логистическая регрессия показали себя хуже: `0.5952` и `0.5873` соответственно.

<a id='8-section'></a>
### 4. Изменение порога

[Вернуться к оглавлению](#0-section)

По умолчанию значение порога равно 0.5. Что, если мы поменяем это значение, может быть наша модель станет лучше?

In [ ]:
# напишем функцию, которая принимает на вход модель и возвращает значение f1-меры и значение порога. 
def get_f1_threshold(model, proba_one_test):
    # переберем значения порогов от 0 до 1 с шагом в 0.01
    best_f1 = 0
    for threshold in np.arange(0, 1, 0.01):
        predicted_test = proba_one_test > threshold
        f1 = f1_score(tar_test_ohe, predicted_test)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    print('Лучшее значение F1-меры - {:.4f} - достигается при значении порога - {:.2f}'.format(best_f1, best_threshold))
    predicted_test = proba_one_test > best_threshold
    return best_f1, best_threshold, predicted_test

In [ ]:
(
    f1_xgb_bal,
    threshold_xgb_bal,
    predicted_test_xgb_bal,
) = get_f1_threshold(rf_model_disb, rf_proba_disb)

In [ ]:
(
    f1_xgb_bal,
    threshold_xgb_bal,
    predicted_test_xgb_bal,
) = get_f1_threshold(xgb_model_bal, xgb_proba_bal)

In [ ]:
(
    f1_xgb_bal,
    threshold_xgb_bal,
    predicted_test_xgb_bal,
) = get_f1_threshold(xgb_model_bal, xgb_proba_bal)

In [ ]:
(
    f1_xgb_bal,
    threshold_xgb_bal,
    predicted_test_xgb_bal,
) = get_f1_threshold(xgb_model_bal, xgb_proba_bal)

In [ ]:
(
    f1_xgb_bal,
    threshold_xgb_bal,
    predicted_test_xgb_bal,
) = get_f1_threshold(xgb_model_bal, xgb_proba_bal)

Таким образом, нам удалось достичь значения F1-меры `0.6586` на валидационной выборке при значении порога - `0.59` для модели CatBoost с помощью техники downsampling.

In [ ]:
df_ctb_down['Значения метрик_threshold'] = [accuracy_score(target_valid, predicted_valid_ctb_down),
                                           recall_score(target_valid, predicted_valid_ctb_down),
                                           precision_score(target_valid, predicted_valid_ctb_down),
                                           f1_score(target_valid, predicted_valid_ctb_down),
                                           roc_auc_score(target_valid, probabilities_one_valid_ctb_down)]

<a id='9-section'></a>
### 5. Визуализация метрик

[Вернуться к оглавлению](#0-section)

In [ ]:
#построим PR-кривую
precision, recall, thresholds = precision_recall_curve(target_valid, probabilities_valid_ctb_down[:, 1])

plt.figure(figsize=(9, 6))
plt.step(recall, precision, where='post')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Кривая Precision-Recall')
plt.scatter(df_ctb_down.loc['recall', 'Значения метрик_threshold'], df_ctb_down.loc['precision', 'Значения метрик_threshold'], \
            color='black', s=40)
plt.annotate('Max F1-score', xy=(df_ctb_down.loc['recall', 'Значения метрик_threshold'], \
                                 df_ctb_down.loc['precision', 'Значения метрик_threshold']), xytext=(0.8, 0.8),
             arrowprops=dict(facecolor='black', shrink=0.05)
             )
plt.show()

In [ ]:
# построим ROC-кривую
# для сравнения на графике представлена ROC-кривая случайной модели
fpr_lgb, tpr_lgb, thresholds_lgb = roc_curve(
    tar_test_oe, lgb_proba_bal)
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(
    tar_test_oe, xgb_proba_bal)
fpr_ctb, tpr_ctb, thresholds_ctb = roc_curve(
    tar_test_oe, ctb_proba_disb)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(
    tar_test_oe, rf_proba_disb)
fpr_lr, tpr_lr, thresholds_lr = roc_curve(
    tar_test_oe, lr_proba_disb)
fpr_dt, tpr_dt, thresholds_dt = roc_curve(
    tar_test_oe, dt_proba_disb)
plt.figure(figsize=(13, 13))
plt.plot(fpr_lgb, tpr_lgb, label='LightGBM')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.plot(fpr_ctb, tpr_ctb, label='CatBoost')
plt.plot(fpr_rf, tpr_rf, label='Случайный лес')
plt.plot(fpr_lr, tpr_lr, label='Логистическая регрессия')
plt.plot(fpr_dt, tpr_dt, label='Решающее дерево')
plt.legend()
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

### Вывод

В результате исследования моделей классификации было выявлено, что лучшая модель для данной задачи - CatBoost. Максимальное значение F1-меры такой модели равно `0.6586`. Оно получается после изменения порога до значения `0.59`.

<a id='9-section'></a>
## V. Общий вывод

В данной работе была получена модель, которая прогнозирует, уйдёт клиент из банка в ближайшее время или нет.

Данная модель - это модель градиентного бустинга CatBoost, которая успешно предсказывает результат примерно в `84.5%` случаев.

Также были исследованы модели логистической регрессии, решающего дерева, случайного леса, LightGBM и XGBoost, но они показали результаты хуже CatBoost.

Изначально при обучении модели на выборке с дисбалансом классов (отрицательных ответов в 4 раза больше положительных) модель случайного леса показала лучшее значение F1-меры - `0.6213`.

После балансировки классов техникой увеличения / уменьшения выборки, значение метрики f1 у случайного леса стало только хуже. А вот CatBoost показал себя хорошо, увеличив значение с `0.5918` до `0.6185`. 

F1-меру удалось поднять благодаря изменению порога до `0.59`. Значение F1-меры увеличилось до `0.6586`.

Также были построены кривые метрик: PR-кривая и ROC-кривая. Значение AUC-ROC `0.8445` говорит о том, что наша модель сильно отличается от случайной, т.к. чем график выше, тем больше значение TPR и, соответственно, лучше качество модели. 

На тестовой выборке модель показала себя хорошо: значение F1-меры равно `0.623`.